# Extract Email Texts
The email-texts contain unwanted information such as signatures and preceding conversations. This is to be extracted.

### Filter DF into sender_access: EX / SMTP / NaN

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_parquet('avocado-1.0.2/avocado_light.parquet', engine='pyarrow')


In [3]:
df.shape

(938035, 14)

In [4]:
df = df.loc[~df.isna().all(axis=1)]

In [5]:
df.shape

(937958, 14)

In [6]:
df_ex = df[df['sender_access'] == 'EX']

#### Sender Type EX

In [7]:
df_ex.shape

(600564, 14)

In [8]:
df_ex['body_content'].str.contains('Message-ID:', case=False).sum()

np.int64(600564)

In [9]:
df_ex.loc[:, 'body_content'] = df_ex['body_content'].fillna('')

# Define the regex pattern
pattern = re.compile(r'message-id:\s*<[^>]+>\s*(.*)', re.IGNORECASE | re.DOTALL)

# Filter DataFrame to get only rows that contain the Message-ID pattern
df_ex = df_ex[df_ex['body_content'].str.contains(r'message-id:\s*<[^>]+>', case=False, regex=True)]

# Extract the text following the Message-ID using .loc to avoid SettingWithCopyWarning
df_ex.loc[:, 'extracted_text'] = df_ex['body_content'].str.extract(pattern, expand=False)


In [10]:
df_ex.head()

,messageid,sender_access,outlook_sender_name,outlook_recipient_name,sentto_address,cc_address,subject,processed_subject,sent_date,arrival_date,body_file_path,body_content,in_reply_to,Keywords,extracted_text
0,<FC38A9406AA4D411AB62009027DE9DA5E6545D@HQEXCH01>,EX,Fortunata Hermoso,Meshele Ko,Meshele Ko,'ray_rahamin@hp.com'; Dan Baca; Elba Linscott,Fw: RE: AvocadoIT's CEO committed to winning B...,AvocadoIT's CEO committed to winning Banamex deal,2001-04-12T22:02:41Z,2001-04-12T22:02:41Z,text/162/162-000001-EM.txt,"From: ""Fortunata Hermoso"" <fortunata.hermoso@a...",None,None,Fyi on Venks commitment on Banamex. The next s...
1,<19B3B310D020D311B57E00105A9A55241D5975@COFFEE>,EX,Paiman Komeilizadeh,Meshele Ko,All Employees,None,Expense reimbursement request- Amendment,Expense reimbursement request- Amendment,2000-04-24T23:06:06Z,2000-04-24T23:06:15Z,text/162/162-000003-EM.txt,"From: ""Paiman Komeilizadeh"" <pkomeilizadeh@avo...",None,None,"Dear friends,\r\n\r\n\r\nTo expedite the expe..."
2,<F57EA917353BD411B5BA00D0B708160C121B24@COFFEE>,EX,Meshele Ko,None,Piyush Goel; Dan Baca,None,FW: Information,Information,2000-06-20T20:51:04Z,2000-06-20T20:51:04Z,text/162/162-000004-EM.txt,"From: ""Meshele Ko"" <mko@avocadoit.com>\r\nTo: ...",None,None,Here is the information from Send.com. They h...
3,<9640A23C9075D411B5CB00D0B708160C23A93D@COFFEE>,EX,Meshele Ko,None,'jbmcbryde@fedex.com',None,Fedex,Fedex,2000-09-12T17:01:08Z,2000-09-12T17:01:08Z,text/162/162-000005-EM.txt,"From: ""Meshele Ko"" <mko@avocadoit.com>\r\nTo: ...",None,None,"Hello Jim,\r\n\r\nI am following up on a piece..."
5,<19B3B310D020D311B57E00105A9A55243C6C85@COFFEE>,EX,Ron Silverton,Meshele Ko,Marketing&Business Development; Sales Team; Ex...,None,m-commerce over $120 billion by 2008,ASG News & Analysis - 6 April 2000,2000-04-10T19:19:56Z,2000-04-10T19:19:57Z,text/162/162-000008-EM.txt,"From: ""Ron Silverton"" <rsilverton@avocadoit.co...",None,None,Attached are some recent projections from Merr...


##### Extract Original Message

In [11]:
df_ex.loc[0]['extracted_text']

'Fyi on Venks commitment on Banamex. The next step is to set a conference call with Nicollo and Venk next week. EP is committed to do whatever it takes to close this accoiunt.\r\n\r\nWe have a lot of support. Let\'s keep the communcation going. Please use me to escalate any support needed to close this account.\r\n--------------------------\r\nFortunata Hermoso\r\nAvocadoIT, Inc.\r\nGlobal Alliance Manager\r\nMobile Phone: 408.464.8998\r\nemail: fortunata.hermoso@avocadoit.com\r\n\r\n\r\n-----Original Message-----\r\nFrom: Venk Shukla <shukla@avocadoit.com>\r\nTo: \'DEPERALTA,AIREEN (HP-Cupertino,ex1)\' <aireen_deperalta@hp.com>; Venk Shukla <shukla@avocadoit.com>; ARENAS,MARTHA (HP-Mexico,ex1) <martha_arenas@hp.com>; SPATARO,NICCOLO (HP-Mexico,ex1) <niccolo_spataro@hp.com>\r\nCC: KOMORI,ZENKI (HP-Mexico,ex1) <zenki_komori@hp.com>; ZAMORA,EDUARDO (HP-Mexico,ex1) <eduardo_zamora@hp.com>; Fortunata Hermoso <Fortunata.Hermoso@avocadoit.com>; WATSON,JULIE (HP-Cupertino,ex1) <julie_watson@h

In [12]:

# Adjust the search string to capture exactly five dashes, spaces, and words
search_string = r"-{5}[\w\s]+-{5}"

# Regex pattern to match and capture text after the search string
pattern = re.compile(rf'{search_string}[\r\n]*(.*)', re.DOTALL)

# Extract text after the search string into 'preceding_conversation'
df_ex['preceding_conversation'] = df_ex['extracted_text'].str.extract(pattern, expand=False)

# Remove the search string and everything after it from 'extracted_text'
df_ex['extracted_text'] = df_ex['extracted_text'].str.replace(pattern, '', regex=True).str.strip()


In [13]:
df_ex.shape

(600564, 16)

In [14]:
df_ex['extracted_text'].str.contains('In-Reply-To').sum()

np.int64(240095)

In [15]:
# Apply the regex pattern to remove matching lines from 'extracted_text' column
df_ex['extracted_text'] = df_ex['extracted_text'].replace(r'^In-Reply-To: <[^>]*>\s*', '', regex=True)

In [16]:
print(df_ex.iloc[0]['extracted_text'])

Fyi on Venks commitment on Banamex. The next step is to set a conference call with Nicollo and Venk next week. EP is committed to do whatever it takes to close this accoiunt.

We have a lot of support. Let's keep the communcation going. Please use me to escalate any support needed to close this account.
--------------------------
Fortunata Hermoso
AvocadoIT, Inc.
Global Alliance Manager
Mobile Phone: 408.464.8998
email: fortunata.hermoso@avocadoit.com


In [17]:
df_ex['extracted_text'].str.contains('In-Reply-To').sum()

np.int64(1)

In [18]:
pattern = re.compile(
    r'(From:.*|Sent:.*|To:.*|Subject:.*)(?:\n|\r\n|$)', re.IGNORECASE | re.MULTILINE
)

# Extract matching lines into 'preceding_conversation'
df_ex['preceding_conversation'] = df_ex['extracted_text'].str.extract(
    rf'((From:.*\n.*\n.*\n.*\n)(.*))', expand=False
)[0]

# Remove matching lines from 'extracted_text'
df_ex['extracted_text'] = df_ex['extracted_text'].str.replace(pattern, '', regex=True).str.strip()


#### Sender Type SMTP

In [19]:
df_smtp = df[df['sender_access'] == 'SMTP']
df_smtp = df_smtp.copy()

In [20]:
df_smtp.shape

(333599, 14)

In [21]:
df_smtp.iloc[0]['body_content']

'Received: from exsrvr.homebid.com (197-175-161-216.customers.uswest.net [216.161.175.197]) by coffee.avocadoit.com with SMTP (Microsoft Exchange Internet Mail Service Version 5.5.2448.0)\r\n\tid J5NF4M7L; Tue, 2 May 2000 18:30:53 -0700\r\nReceived: by EXSRVR with Internet Mail Service (5.5.2448.0)\r\n\tid <JGX7RSRQ>; Tue, 2 May 2000 18:27:20 -0700\r\nMessage-ID: <019D8990549CD3118A30009027D609CD25BF69@EXSRVR>\r\nFrom: Cindy Hilker <chilker@homebid.com>\r\nTo: "Meshele Ko (E-mail)" <mko@avocadoit.com>\r\nCc: Wayne Thayer <wthayer@homebid.com>\r\nSubject: Icon and graphic for our WAP APPs\r\nDate: Tue, 2 May 2000 18:27:13 -0700 \r\nMIME-Version: 1.0\r\nX-Mailer: Internet Mail Service (5.5.2448.0)\r\nContent-Type: multipart/alternative;\r\n\tboundary="----_=_NextPart_001_01BFB49E.B93D8574"\r\n\r\nThis message is in MIME format. Since your mail reader does not understand\r\nthis format, some or all of this message may not be legible.\r\n\r\n------_=_NextPart_001_01BFB49E.B93D8574\r\nConte

In [22]:
pattern_1 = r'Received.*?\\r\\n\\r\\n'

# Apply the regex pattern to 'body_content' column and replace matches with an empty string
df_smtp['body_content'] = df_smtp['body_content'].apply(repr)
df_smtp.loc[:, 'extracted_text'] = df_smtp['body_content'].str.replace(pattern_1, '', regex=True)


In [23]:
def extract_email(row):
    try:
        if 'Content-Transfer-Encoding:' in row['extracted_text']:
            if '--============newsletter============' in row['extracted_text']:
                pattern = r"Contact \\r\\n\\r\\n(.*).*'"
                res = re.search(pattern, row['extracted_text'])
                row['extracted_text'] = res.group(1)
            pattern = r"--\\\\r\\\\n\\\\r|--\\r\\n\\r(.*)"
            res = re.search(pattern, row['extracted_text'])
            row['extracted_text'] = res.group(1)
    except Exception as e:
        row['problematic'] = True
    return row

In [24]:
df_smtp = df_smtp.apply(extract_email, axis=1)

In [25]:
df_smtp.head()

,Keywords,arrival_date,body_content,body_file_path,cc_address,extracted_text,in_reply_to,messageid,outlook_recipient_name,outlook_sender_name,problematic,processed_subject,sender_access,sent_date,sentto_address,subject
4,None,2000-05-03T01:30:54Z,'Received: from exsrvr.homebid.com (197-175-16...,text/162/162-000006-EM.txt,Wayne Thayer,\nMeshele: \r\n\r\nI forgot to mention in our ...,None,<019D8990549CD3118A30009027D609CD25BF69@EXSRVR>,Meshele Ko,Cindy Hilker,NaN,Icon and graphic for our WAP APPs,SMTP,2000-05-03T01:27:13Z,Meshele Ko (E-mail),Icon and graphic for our WAP APPs
7,None,2000-02-01T23:47:19Z,'Received: from office.violet.com (216.186.186...,text/162/162-000010-EM.txt,None,\nHi Meshele~ \r\n \r\n \r\nThanks for your ...,None,<57F6D4070FCAD3119D8F00C04F10A7C2058196@EXCH01>,Meshele Ko,Violet Customer Service,NaN,Non-PC access to Web application,SMTP,2000-02-01T23:38:46Z,'Meshele Ko',RE: Non-PC access to Web application
8,None,2000-08-08T01:16:24Z,'Received: from mail02-oak.pilot.net (mail-oak...,text/162/162-000013-EM.txt,vasuki_narayan@peoplesoft.com,"\nMeshele,\r\n\r\nThanks again for coming to P...",None,<OFA65BD9FB.792EF8BC-ON88256935.0004FEA0@peopl...,Meshele Ko,diana_smith@peoplesoft.com,NaN,AvocadoIT Demo,SMTP,2000-08-08T01:05:59Z,mko@avocadoit.com,AvocadoIT Demo
9,None,2000-12-07T22:00:03Z,'Received: from mailer.avocadoit.com (10.7.1.4...,text/162/162-000015-EM.txt,None,"'Meshele, \r\n\r\nI have been informed that th...",None,<55900733E95DD2119A7400A0C9975CD70353C83D@MSGT...,Meshele Ko,"Knox, Ron",NaN,Signed SOW,SMTP,2000-12-07T21:38:55Z,'Meshele Ko',Signed SOW
14,None,2000-03-06T21:30:18Z,'Received: from MAILSERVER (206.57.35.4 [206.5...,text/162/162-000023-EM.txt,Keylan Qazzaz,"'Hi Meshele,\r\n\r\nAs you know I will be out ...",None,<NCBBJBIBCLLIOIJIMPCHCEDECAAA.johnn@theplunge....,Meshele Ko,John Nunez,NaN,While I'm Away,SMTP,2000-03-06T21:24:47Z,Meshele Ko,While I'm Away


In [26]:
df_smtp.iloc[3]['extracted_text']

"'Meshele, \\r\\n\\r\\nI have been informed that the signed SOW has now been faxed back to\\r\\nAvocadoIT. Please call me if you did not receive it.\\r\\n\\r\\nRegards, Ron.\\r\\n\\r\\n> Ron Knox\\r\\n> Project Manager, e.Business Division\\r\\n> Fidelity Investments\\r\\n> ron.knox@fidelity.com\\r\\n> 416.217.7599\\r\\n> \\r\\n>\\r\\n'"

#### Sender Type not available

In [27]:
df_nan = df[(df['sender_access'] != 'SMTP' ) & (df['sender_access'] != 'EX') ]

In [28]:
df_nan.shape

(3795, 14)

In [29]:

# Drop columns where all values are None


df_nan['body_content'] = df_nan['body_content'].apply(repr)


/var/folders/vh/4lm899sj0tdgnf9d98kzk1800000gn/T/ipykernel_38209/2766733130.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nan['body_content'] = df_nan['body_content'].apply(repr)


In [30]:
df_nan.shape

(3795, 14)

In [31]:
df_nan_w_message = df_nan[df_nan['body_content'].str.contains('Message-ID:')]
df_nan_w_message.shape[0]

3588

In [32]:
df_nan_wo_message = df_nan[~df_nan['body_content'].str.contains('Message-ID:')]
df_nan_wo_message.shape[0]

207

In [33]:
# make sure that entries with and without having "message id" in their body content still adds up to the amount of rows of the original df
if df_nan[df_nan['body_content'].str.contains('Message-ID:')].shape[0] + df_nan_wo_message.shape[0] != df_nan.shape[0]:
    raise Exception

In [34]:
df_nan_wo_message.iloc[115]['body_content']

'\'From: "Marcia Kadanoff" <marciak@avocadoit.com>\\r\\nTo: "Venk Shukla" <shukla@avocadoit.com>\\r\\nCc: "Pam Fong" <pfong@avocadoit.com>\\r\\nSubject: Marketing Organization\\r\\nDate: 04 Jan 2000 02:50:58 UTC\\r\\n\\r\\nProposal on the table - as shared with you today\\r\\n \\r\\nMarcia Kadanoff\\r\\nChief Marketing Officer\\r\\nAvocadoIT\\r\\n\\r\\n650.270.4309\\r\\nwww.avocadoit.com\\r\\n\''

In [35]:
def extract_email_nan(row):
    try:
        row['extracted_text'] = row['body_content']
        if 'Date:' in row['body_content']:
            pattern = r'(Date:.*?\\r\\n\\r\\n|Date:.*?\\\\r\\\\n\\\\r\\\\n)(.*)'
            res = re.search(pattern, row['body_content'])
            row['extracted_text'] = res.group(2)
    except:
        print('exception happened')
        row['problematic'] = True
    return row

In [36]:
df_nan_wo_message = df_nan_wo_message.apply(extract_email_nan, axis=1)

In [37]:
df_nan_wo_message.iloc[100]['extracted_text']

"When I chatted with Joe G. he mentioned he was about to  make an offer to someone to handle Europe...\\r\\n\\r\\nmk\\r\\n\\r\\nMarcia Kadanoff\\r\\nChief Marketing Officer\\r\\nAvocadoIT\\r\\n\\r\\n650.270.4309\\r\\nwww.avocadoit.com\\r\\n'"

In [38]:
df_nan_wo_message.shape

(207, 15)

In [39]:
df_nan_w_message.loc[:, 'body_content'] = df_nan['body_content'].fillna('')

# Define the regex pattern
pattern = re.compile(r'message-id:\s*<[^>]+>\s*(.*)', re.IGNORECASE | re.DOTALL)

# Filter DataFrame to get only rows that contain the Message-ID pattern
df_nan_w_message = df_nan_w_message[df_nan_w_message['body_content'].str.contains(r'message-id:\s*<[^>]+>', case=False, regex=True)]

# Extract the text following the Message-ID using .loc to avoid SettingWithCopyWarning
df_nan_w_message.loc[:, 'extracted_text'] = df_nan_w_message['body_content'].str.extract(pattern, expand=False)


In [40]:
df_nan_w_message.shape

(3588, 15)

In [41]:
df_nan_wo_message.shape

(207, 15)

In [42]:
df_nan_new = pd.concat([df_nan_w_message, df_nan_wo_message], axis=0)

In [43]:
df_nan_new.shape

(3795, 15)

In [44]:
df_final = pd.concat([df_ex, df_smtp, df_nan_new])

In [45]:
df_final.shape

(937958, 17)

In [46]:
df_final.shape

(937958, 17)

In [61]:
df_final.to_parquet('avocado_extracted_text.parquet', engine='pyarrow')